In [1]:
import pickle
from datasets import load_dataset
import random

# KorQuAD 데이터셋 로드
dataset = load_dataset("KorQuAD/squad_kor_v1")

# 상위 10개 샘플만 선택
subset = dataset['train'].select(range(10))

# 데이터 준비 (상위 10개만 선택)
questions = [sample['question'] for sample in subset]
contexts = [sample['context'] for sample in subset]
answers = [sample['answers']['text'][0] for sample in subset]

# 각 질문에 대해 여러 개의 Context 생성 (정답 포함 + 무작위 Context 추가)
num_contexts = 3  # 각 질문에 대해 3개의 Context 설정
all_contexts = []
for i in range(len(questions)):
    context_list = [contexts[i]]  # 정답이 포함된 실제 Context
    for _ in range(num_contexts - 1):  # 나머지 Context는 무작위로 추가
        random_context = random.choice(contexts)
        context_list.append(random_context)
    all_contexts.append(context_list)

# RAG 데이터 구조 생성
rag_data = {
    'questions': questions,
    'contexts': all_contexts,
    'contexts_answer_idx': [0] * len(questions),  # 실제 답변이 포함된 Context 인덱스를 첫 번째로 설정
    'answers': answers
}

# pickle로 데이터 저장
with open('korquad_rag_data_10_new.pkl', 'wb') as f:
    pickle.dump(rag_data, f)


c:\Users\Byung Mu Kang\RAG-1stWeek\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(questions)

10

In [3]:
for question in questions[:5]:
    print(question)

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?
바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?
1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?
파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?


In [4]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))



In [ ]:
from tqdm import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

predictions = []

# 각 질문에 대해 OpenAI 모델을 호출하여 예측 응답을 생성
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in reading comprehension and Korean general knowledge. Please answer the following question using only a single word. Use information given in Context appropriately.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI 모델 호출하여 응답 생성
    predictions.append(prediction)  # 생성된 응답을 predictions 리스트에 추가
